In [23]:
import cv2
import numpy as np
import imageio as io
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import normalize,StandardScaler
import numpy as np
import os
import random
from sklearn.model_selection import train_test_split
import skimage.io as io
from skimage.feature import local_binary_pattern
from skimage import color
from operator import eq
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import pandas as pd

In [2]:
import pandas as pd

In [ ]:
emotion_data = pd.read_csv('fer2013.csv ')
emotion_data_new = pd.read_csv('fer2013new.csv')
X_train = []
y_train = []
X_test = []
y_test = []
emotions=['neutral','happiness','surprise','sadness','anger','disgust','fear','contempt']
for (index, row), (_, row_n) in zip(emotion_data.iterrows(),emotion_data_new.iterrows()):
#     print(row['Usage'])
    k = row['pixels'].split(" ")
    if row_n['Usage'] == 'Training':
          for j in range(len(emotions)):
                if(row_n[emotions[j]]>=5):
                    X_train.append(np.array(k))
                    y_train.append(j)
    elif row_n['Usage'] == 'PublicTest':
        for j in range(len(emotions)):
            if(row_n[emotions[j]]>=5):
                X_test.append(np.array(k))
                y_test.append(j)
for (index, row), (_, row_n) in zip(emotion_data.iterrows(),emotion_data_new.iterrows()):
    k = row['pixels'].split(" ")
    if row_n['Usage'] == 'Training':
        for j in range(len(emotions)):
            if(row_n[emotions[j]]>=5):
                X_train.append(np.array(k))
                y_train.append(j)
    elif row_n['Usage'] == 'PublicTest':
        X_test.append(np.array(k))
        y_emotions=[]
        for j in range(len(emotions)):
            if(row_n[emotions[j]]!=0):
                y_emotions.append(j)
        y_test.append(y_emotions)
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)
print(X_test.shape)

#################################################
# X_train =np.reshape(X_train,(147, 350))
# X_test =np.reshape(X_test,(147, 350))

X_train = X_train.reshape(X_train.shape[0],48, 48, 1)
X_test = X_test.reshape(X_test.shape[0],48, 48, 1)
X_train=X_train.reshape((len(X_train),48,48))
X_test=X_test.reshape((len(X_test),48,48))
################################################
X_test = X_test.astype(np.float)
#y_test = y_test.astype(np.integer)
X_train = X_train.astype(np.float)
y_train = y_train.astype(np.integer)        
test_labels=y_test
train_labels=y_train
train_features=[feature_extract(img,8) for img in X_train]
train_features=np.array(train_features)

test_features=[feature_extract(img,8) for img in X_test]
test_features=np.array(test_features)

In [1]:

def CodeToValue(code,count):
    binary=[1,2,4,8,16,32,64,128]
    codetovalue=0
    for i in range(count-1):
        codetovalue=codetovalue+code[i]*binary[i]
    return codetovalue    

def bottomlbp(block,x,y):
    bottomdlbp=[0,0,0,0]
    four=block[x-1][y]
    six=block[x+1][y]
    seven=block[x-1][y-1]
    eight=block[x][y-1]
    nine=block[x+1][y-1]

    if(four>=eight):
        bottomdlbp[0]=1
    if(seven>=eight):
        bottomdlbp[1]=1
    if(nine>=eight):
        bottomdlbp[2]=1
    if(six>=eight):
        bottomdlbp[3]=1        
    return  bottomdlbp   


def toplbp(block,x,y):
    topdlbp=[0,0,0]
    one=block[x-1][y+1]
    two=block[x][y+1]
    three=block[x+1][y+1]
    five=block[x][y]
    
    if(one>=two):
        topdlbp[0]=1
    if(five>=two):
        topdlbp[1]=1
    if(three>=two):
        topdlbp[2]=1
    return  topdlbp     

def histogram(block,size):
    hist,bins =np.histogram(block, bins=size, range=(0, size))
    return hist,bins

def DoubleLocalBinaryPatternBlock(block):

    tophist=[]
    bottomhist=[]
    block=padding(block,1)
    w,l=block.shape
    blockcopy = np.zeros((w,l), np.uint8)

    
    for y in range(w-1):
        for x in range(l-1):

            tdlbpcode=toplbp(block,x,y)
            bdlbpcode=bottomlbp(block,x,y)
            tdlbpvalue=CodeToValue(tdlbpcode,3)
            bdlbpvalue=CodeToValue(bdlbpcode,4)
            
            tophist.append(tdlbpvalue)
            bottomhist.append(bdlbpvalue)
#     print(np.array(tophist)  )
    return np.array(tophist) ,np.array(bottomhist) 

def DoubleLocalBinaryPattern(img,block_size):
    w,l=img.shape
    window=w//block_size
    tophistlist=[]
    bottomhistlist=[]
    for r in range(0,w,window):
        for c in range(0,l,window):
            block = img[r:r+window,c:c+window]
            tophist,bottomhist=DoubleLocalBinaryPatternBlock(block)
            th,_=histogram(tophist,8)
            bh,_=histogram(bottomhist,16)
            tophistlist.append(np.array(th))
            bottomhistlist.append(np.array(bh))
    feature=[]
#     feature.append(tophistlist)
#     feature.append(bottomhistlist)
#     print(np.array(tophistlist))
    feature=np.concatenate((np.array(tophistlist), np.array(bottomhistlist)), axis=None)
    return np.array(feature)


In [2]:
#Get the local binary pattern histogram
def local(image):
    lbp = local_binary_pattern(image, 24,3, method="uniform")
    (hist, _) = np.histogram(lbp.ravel(),bins=np.arange(0, 256),range=(0, 256))
    hist = hist.astype("float")
    return hist

In [3]:
#calculate whether the gc greater than alpha set it by 1 otherwise set by -1
def f1_alpha(gc,alpha1):
    if((gc-alpha1)>=0):
        return 1
    else:
        return -1    

In [4]:
#calculate whether the (gc-alpha1)*(gc-alpha2) greater than zero set it by 1 otherwise set by -1
def f2_alpha(gc,alpha1,alpha2):
    if(((gc-alpha1)*(gc-alpha2)) >=0):
        return 1
    else:
        return -1

In [5]:
#Get the pixels that lie in level1
def level1(gc,block,x,y):
    l1=np.zeros((8,), np.uint8)
    l1=[block[x-1][y+1],block[x][y+1],block[x+1][y+1],block[x-1][y],block[x+1][y],block[x-1][y-1],block[x][y-1],block[x+1][y-1]]
    return l1

In [6]:
#Get the pixels that lie in level2
def level2(gc,block,x,y):
    l2=np.zeros((16,1), np.uint8)
    l2=[   block[x-2][y+2],block[x][y+1],block[x+1][y+1],block[x][y+1],  block[x+2][y+2],
           block[x-2][y+1],                                              block[x+2][y+1],
           block[x-2][y],                                                block[x+2][y],
           block[x-2][y-1],                                              block[x+2][y-1],
           block[x-2][y-2],block[x-1][y-2],block[x][y-2],block[x+1][y-2],block[x+2][y-2]
        ]

    return l2

In [7]:
#Calculate the Taylor Feature Map : when each pixel mapped to Taylor domain , we could tell that we get the TFM
def TFM(gc,block,x,y):
#first level
    lev1=level1(gc,block,x,y)
#second level
    lev2=level2(gc,block,x,y)
#the mean gray value of all the pixels in the second-order TU2
#     f0=np.mean(np.array(block).flatten())
#the mean gray value of all the pixels in the first layer        
    alpha1=np.mean(lev1)
#the mean gray value of all the pixels in the second layer        
    alpha2=np.mean(lev2)
#the mean gray value of all the pixels in the first layer and the second layer 
    f0=alpha=np.mean(np.concatenate((np.array(lev1), np.array(lev2)), axis=None))
#The second-order pixel feature    
    f2_gc=f0+f1_alpha(gc,alpha)*(gc-alpha)+f2_alpha(gc,alpha1,alpha2)*((gc-alpha)**2)
    return f2_gc

In [8]:
#Divides the image into NXN blocks , then calculate the TFM for each block and return this TFM block
def TFM_i(img,block_size):
    w,l=img.shape
    window=w//block_size
    TFP_hist=[]
    tfmcopy = np.zeros((w,l), np.uint8)
    for r in range(0,w-2,1):
        for c in range(0,l-2,1):
            gc=img[c][r]
            tfmcopy[c][r]=TFM(gc,img,c,r)
    return tfmcopy

In [9]:
#Calculate the histogram of every block and then concatenate all these histograms and get our feature vectors
def TFM_hist(img,block_size):
    w,l=img.shape
    window=w//block_size
    TFP_hist=[]
    img=padding(img,2)
    for r in range(2,w-2,window):
        for c in range(2,l-2,window):
            block = img[r:r+window,c:c+window]
            tfmh=local(block)
            TFP_hist.append(np.array(tfmh))            
    feature=np.array(TFP_hist).flatten()
    return feature

In [10]:
#This is our pipline, first we get the TFM then get the histogram
def feature_extract(img,blocksize):
    t=TFM_i(img,blocksize)
    ht=TFM_hist(t,blocksize)
    return ht

In [11]:
#pad input block to specific size
def padding(block,size):
    padd=np.pad(block, (size, size), 'constant', constant_values=(0, 0))
    return padd

In [12]:
#Get the histogram 
def histogram(block,size):
    hist,bins =np.histogram(block, bins=size, range=(0, size))
    return hist,bins

In [13]:
#Calculate the accuracy 
def acc(y_pred,test_labels):
    le = preprocessing.LabelEncoder()
    yt = le.fit_transform(y_pred)
    rt = le.fit_transform(test_labels)
    eqw = sum(map(eq, list(rt), list(yt)))
    size=len(rt)
    acc=(eqw/size)*100
    return acc

In [14]:
def load_dataset(feature_set='tse'):
    features = []
    labels = []
    for i, fn in enumerate(img_filenames):
            label = fn.split('.')[0]
            #labels.append(label)
            path = os.path.join(path_to_dataset, fn)
            subpath=os.listdir(path)
            #print(subpath)
            for k, d in enumerate(subpath):             
                labels.append(label)
                imgpath = os.path.join(path, d)
                img = io.imread(imgpath)
                resized = cv2.resize(img, (48,48), interpolation = cv2.INTER_AREA)
                gray=color.rgb2gray(resized)
                logI=Log_Transformation(gray)
                lapI=laplacian_Transformation(logI)
                features.append(np.array(feature_extract(lapI,16)))
                if k > 0 and k % 10== 0:
                    print("[INFO] processed {}/{}".format(k, len(subpath)))
    return features, labels            

In [15]:
def extract_features(img, feature_set='lbp'):
    if feature_set=='lbp':
        return DLBP(img) 
    if feature_set=='tse':
        return Taylor_Exp(img)

In [16]:
def split_data(features,labels,precentage):
    train_features, test_features, train_labels, test_labels = train_test_split(
        features, labels, test_size=0.2, random_state=random_seed)
    return train_features, test_features, train_labels, test_labels

In [17]:
def normalization(train_features, test_features):
    scaler = StandardScaler()
    scaler.fit(train_features)

    train_features = scaler.transform(train_features)
    test_features = scaler.transform(test_features)
    return train_features, test_features

In [18]:
def KNN_classifier(kcount,train_features, train_labels):
    classifier = KNeighborsClassifier(n_neighbors=kcount)
    classifier.fit(list(train_features),list(train_labels) )
    return classifier

In [19]:
def test_classifier(test_features, test_labels,classifier):
        accuracy = classifier.score(test_features, test_labels)        
        print('accuracy:', accuracy*100, '%')

In [20]:
def Log_Transformation(image):
    c = 255 / np.log(1 + np.max(image))+1 
    s = c * (np.log(image + 1)) 
    s = np.array(s, dtype = np.uint8)
    return s

In [21]:
def laplacian_Transformation(image):
    im = cv2.Laplacian(image,cv2.CV_64F)
    return im

In [24]:
# path_to_dataset =r'E:\datasetemotion\val\val'
# path_to_dataset =r'E:\datasetemotion\train\train'
# path_to_dataset =r'D:\Download\jaffedbase'
path_to_dataset =r'E:\emotion\archive\ck\CK+48'
img_filenames = os.listdir(path_to_dataset)

In [25]:
img_filenames = os.listdir(path_to_dataset)
print(img_filenames)

['anger', 'contempt', 'disgust', 'fear', 'happy', 'sadness', 'surprise']


In [26]:
random_seed = 0  
random.seed(random_seed)
np.random.seed(random_seed)

In [ ]:
print('Loading dataset. This will take time ...')
features, labels = load_dataset(feature_set='tse')
print('Finished loading dataset.')

Loading dataset. This will take time ...


E:\Anaconda\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


[INFO] processed 10/135
[INFO] processed 20/135
[INFO] processed 30/135
[INFO] processed 40/135
[INFO] processed 50/135
[INFO] processed 60/135
[INFO] processed 70/135
[INFO] processed 80/135
[INFO] processed 90/135
[INFO] processed 100/135
[INFO] processed 110/135
[INFO] processed 120/135
[INFO] processed 130/135


In [ ]:
features1, labels1=features, labels

In [ ]:
train_features, test_features, train_labels, test_labels=split_data(np.array(features1),np.array(labels1),0.2)

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=0.7)
X_train1 = pca.fit_transform(train_features)
X_test1 = pca.transform(test_features)
# n_components=1000 must be between 0 and min(n_samples, n_features)=900 with svd_solver='full'
train_features=X_train1
test_features=X_test1
train_labels=train_labels
test_labels=test_labels

In [ ]:
classifier=KNN_classifier(3,train_features, train_labels)

In [ ]:
y_pred = classifier.predict(test_features)

In [ ]:
accuracy = classifier.score(test_features, test_labels)
print('Test accuracy:', accuracy*100, '%')
accuracyt = classifier.score(train_features, train_labels)
print('Train accuracy:', accuracyt*100, '%') 

# Test accuracy: 45.17766497461929 %
# Train accuracy: 71.68367346938776 %
# Test accuracy: 56.852791878172596 %
# Train accuracy: 84.0561224489796 %
#without LL
# Test accuracy: 59.390862944162436 %
# Train accuracy: 85.96938775510205 %
#with LL
# Test accuracy: 58.37563451776649 %
# Train accuracy: 86.86224489795919 %
# block size =16
# Test accuracy: 68.02030456852792 %
# Train accuracy: 94.13265306122449 %


In [ ]:
from sklearn.metrics.pairwise import rbf_kernel
le = preprocessing.LabelEncoder()
random_seed = 0  
random.seed(random_seed)
np.random.seed(random_seed)
sigma=0.0008
c=64
reg=100
err=1e-4
def kernel(X, Y=None,g=None):
    k=rbf_kernel(X, Y, gamma=g)
    return k
def KELM_Beta(sigma,c,x,y):
    le.fit(y)
    y = le.transform(y)
    emotion = len(np.unique(y))
    One_Hot_Encoding=np.zeros((emotion,emotion), np.uint8)
    for em in range (emotion):
        One_Hot_Encoding[em][em]=1
    T = One_Hot_Encoding[y, :]    
    N,d=x.shape
    Omega=kernel(x,None,sigma)
    I=np.eye(N)
    Beta = np.linalg.inv((I /c) + Omega).dot(T)    
    return Beta,One_Hot_Encoding
def KELM_Output(sigma,x,x_test,Beta,One_Hot_Encoding):
    hx=kernel(x_test,x,sigma)
    fx= hx.dot(Beta)
#     print(np.array(fx).shape)
    p=np.argmax(fx,axis=1)
    return p
def acc(y_pred,test_labels):
    le = preprocessing.LabelEncoder()
    yt = le.fit_transform(y_pred)    
    rt = le.fit_transform(test_labels)
    eqw = sum(map(eq, list(rt), list(y_pred)))
    size=len(rt)
    acc=(eqw/size)*100
    return acc
Beta,One_Hot_Encoding=KELM_Beta(sigma,c,np.array(train_features),np.array(train_labels))
p1=KELM_Output(sigma,np.array(train_features),np.array(test_features),Beta,One_Hot_Encoding)
print("test",acc(p1,test_labels))

p2=KELM_Output(sigma,np.array(train_features),np.array(train_features),Beta,One_Hot_Encoding)
print("train",acc(p2,train_labels))

In [ ]:
print(accuracy_score(test_labels, y_pred))
print(classification_report(test_labels, y_pred))
print(confusion_matrix(test_labels, y_pred))

In [ ]:
from sklearn.svm import SVC
# k in ['rbf','sigmoid']\
# gamma in ['scale','auto']

k='rbf'
gamma = 'scale'
c=900
clf = SVC(C=c,kernel=k,gamma=gamma)
clf.fit(train_features, train_labels)

print ('Test score', clf.score(test_features, test_labels)*100 )
print ('Train score', clf.score(train_features, train_labels)*100 )



# Test score 43.14720812182741
# Train score 52.67857142857143
#withoutLL
# Test score 55.32994923857868
# Train score 100.0
#with LL
# Test score 59.390862944162436
# Train score 100.0
# block size =16
# Test score 69.03553299492386
# Train score 100.0


In [36]:
from sklearn.metrics.pairwise import rbf_kernel
le = preprocessing.LabelEncoder()
random_seed = 0  
random.seed(random_seed)
np.random.seed(random_seed)
sigma=0.0008
c=64
reg=100
err=1e-4

def kernel(X, Y=None,g=None):
    k=rbf_kernel(X, Y, gamma=g)
    return k

def KELM_Beta(sigma,c,x,y):
    le.fit(y)
    y = le.transform(y)
    emotion = len(np.unique(y))
    One_Hot_Encoding=np.zeros((emotion,emotion), np.uint8)
    for em in range (emotion):
        One_Hot_Encoding[em][em]=1
    T = One_Hot_Encoding[y, :]    
    N,d=x.shape
    Omega=kernel(x,None,sigma)
    I=np.eye(N)
    Beta = np.linalg.inv((I /c) + Omega).dot(T)    
    return Beta,One_Hot_Encoding

def KELM_Output(sigma,x,x_test,Beta,One_Hot_Encoding):
    hx=kernel(x_test,x,sigma)
    fx= hx.dot(Beta)
    print(np.array(fx).shape)
    p=np.argmax(fx,axis=1)
    return p

def acc(y_pred,test_labels):
    le = preprocessing.LabelEncoder()
    yt = le.fit_transform(y_pred)    
    rt = le.fit_transform(test_labels)
    eqw = sum(map(eq, list(rt), list(y_pred)))
    size=len(rt)
    acc=(eqw/size)*100
    return acc

Beta,One_Hot_Encoding=KELM_Beta(sigma,c,np.array(train_features),np.array(train_labels))
p=KELM_Output(sigma,np.array(train_features),np.array(test_features),Beta,One_Hot_Encoding)
print("test",acc(p,test_labels))

p=KELM_Output(sigma,np.array(train_features),np.array(train_features),Beta,One_Hot_Encoding)
print("train",acc(p,train_labels))

(197, 7)
test 71.57360406091371
(784, 7)
train 100.0


In [47]:
y=clf.predict(test_features)

In [48]:
print(accuracy_score(test_labels, y))
print(classification_report(test_labels, y))
print(confusion_matrix(test_labels, y))

0.5939086294416244
              precision    recall  f1-score   support

       anger       0.40      0.60      0.48        20
    contempt       0.45      0.83      0.59         6
     disgust       0.61      0.41      0.49        41
        fear       0.50      0.27      0.35        15
       happy       0.62      0.79      0.70        39
     sadness       0.50      0.14      0.21        22
    surprise       0.70      0.83      0.76        54

    accuracy                           0.59       197
   macro avg       0.54      0.55      0.51       197
weighted avg       0.59      0.59      0.57       197

[[12  0  1  1  3  1  2]
 [ 0  5  0  1  0  0  0]
 [ 4  2 17  1  9  0  8]
 [ 3  0  0  4  1  2  5]
 [ 2  1  4  0 31  0  1]
 [ 8  2  1  1  4  3  3]
 [ 1  1  5  0  2  0 45]]


In [49]:
# /happy
# img = io.imread('MK.HA1.116.tiff')
# /angry
# img = io.imread('KA.AN3.41.tiff')
# /sad
# img = io.imread('KA.SA1.33.tiff')
img = io.imread('S010_006_00000013.png')

# img  = io.imread('KA.AN2.40.tiff')
# img  = io.imread('MK.HA1.116.tiff')
# img  = io.imread('UY.DI2.150.tiff')

# img = io.imread('p1.jpg')

# img  = io.imread('KA.AN2.40.tiff')
f=feature_extract(img,8)
print(f)

[8. 3. 0. ... 0. 0. 0.]


In [340]:
# f=extract_features(img,'hog')
fp= pca.transform([f])
# print(fp)
print ('pred label', clf.predict(fp) )


# pca = PCA(n_components=0.9)
# fd = pca.fit_transform([f])
# X_test1 = pca.transform(test_features)


pred label ['happy']
